#  Comparação da Procura no Puzzle dos 2 Colares
## Teste de Avaliação Contínua nº 1
### Introdução à Inteligência Artificial edição 2020/21
#### Data limite de entrega: 18 de Dezembro (1m para a meia noite)


<img src="finalDoisColares.PNG" alt="Drawing" style="width: 250px;"/>

## Contéudo

    1. Vamos mostrar um exemplo de implementação em python do puzzle dos 2 colares, verificando que essa formulação está correcta ao submetê-la a um conjunto de testes.
    2. Vamos comparar em termos de complexidade temporal e espacial diferentes algoritmos: 3 variantes da largura-primeiro, o greedy e o A* para diferentes configurações iniciais, e usaremos duas heurísticas: h1(x) e h2(x)
    3. Será pedido que apenas implementem uma terceira heurística e que comparem o desempenho do A* e Greedy para problemas iniciais a 14 e 16 passos da solução.



|     *       |      *      |
| ----------- | ----------- |
|     Touro no labirinto      |         Formato XXL      |
| <img src="touro-lab.png" alt="Drawing" style="width: 150px;"/>  | Esta avaliação tem um formato XXL, especialmente em termos de tempo, com um objectivo pedagógico, em que terão de percorrer o labirinto até encontrar e vencer o Touro, na verdade é só um tourinho.        |

## Modelização em Python

### Representação dos Estados
Vamos representar os estados como tuplos de 2 colares: o primeiro da esquerda e depois o da direita. Os colares serão listas com os símbolos das cores: na posição 0 teremos a cor da bola das 12h e todas as seguintes se sucedem pela ordem horária. Esta modelização dos estados permite lidar só com 2 colares com 20 bolas e com um número de 4 cores: 'e' para encarnado, 'r' para roxo, 'a' para azul e 'v' para verde. 

Como desejamos usar os algoritmos de procura em grafo, é necessário termos estados *hashable* e como isso não acontece naturalmente com os tuplos com duas listas, teremos de criar uma subclasse de tuple que terá um método associado __hash()__. 

### Subclasse de Problema
#### Informação estática
O problema precisa de ter mais informação que é estática, como por exemplo, quais as posições nas listas dos colares com bolas partilhadas. Sabemos que entre dois colares podemos ter duas bolas partilhadas, nos pares de posições (3,17) e (7,13). A bola da posição 3 do colar da esquerda é a mesma que a bola da posição 17 do colar da direita. E a bola da posição 7 do colar da esquerda é a mesma que a bola da posição 13 do colar da direita. Esses pares ficam guardados nos parâmetros **fundo** e **topo**.

Guardamos como informação estática também o número de bolas de cada colar: 20 e também uma tabela a indicar a posição de cada colar no tuplo só para mapear as acções nos indices do tuplo que representa o estado.

Para testar a validade do estado inicial, no construtor da instância de Problem apenas verificamos se todos os colares têm 20 bolas, se partilham bolas e se existem tantas cores quanto o dobro dos colares e se há 2 cores com 10 bolas e as restantes com 9.

#### goal_test()
Este método não assume que já conhecemos os estados finais. Assim, testa mesmo se existem em cada um dos colares 2 cores ininterruptas de 10 e de 9 bolas.

#### Acções
Aas acções são strings "colar sentido". O colar pode ser esq ou dir.
O sentido pode ser 'h' para horário e 'ah' para anti-horário.

    'esq h' quer dizer que rodo no sentido horário o colar da esquerda, o primeiro.

Assim, precisamos de um dicionário que indique que o colar esquerdo é o índice 0 do tuplo e o colar direito é o índice 1. Poderíamos ter usado um namedtuple para evitar isso, mas não aconteceu.
Notem que o método actions devolve sempre as 4 acções, por esta ordem: "esq h","esq ah", "dir h","dir ah".

#### result()
O método **result()** cria uma deepcópia do duplo com as 2 listas, faz o shift circular correspondente à rotação do colar respetivo e depois actualiza as posições de bolas partilhadas na outra lista correspondente ao colar que não roda. 

#### path_cost()
Não é necessário mexer no ***path_cost()***, todos os custos são unitários.

#### Display do estado

<img src="inicialDoisColares.PNG" alt="Drawing" style="width: 250px;"/>

Eis a representação visual do estado correspondente às configurações dos dois colares da figura anterior:

Dispomos as duas fitas, a de cima é a do colar esquerdo e a de baixo o do direito. t indica a posição nos dois colares da bola partilhada de topo e f indica a posição partilhada do fundo. 

      t   f            
    vvvvvavvrreeeeeeeeev
    aaaaaaarrrrrrvvrrvva
                 f   t  



## A classe Colaris para o estado e a classe PuzzleColares

In [ ]:
from searchPlus import *
from collections import namedtuple, Counter
from copy import deepcopy
import timeit

In [ ]:


class Colaris(tuple):
        def __hash__(self):
            return hash(str(self))

        
class PuzzleColares(Problem):
    """Os dois colares"""

    
    def __init__(self, initial=None,goal=None):
        """Construtor....... que também verifica a validade dos argumentos
        Recebe duas listas de cores, (das 12h no sentido horário), um tuplo a indicar quais os índices nas duas listas
        da bola partilhada de topo e o mesmo com a de fundo.
        terão de ser 4 cores, há 9 bolas com duas das cores e as cores 
        aparecem em 10 bolas, a bola no topo tem de ser a mesma em ambos 
        os anéis e o mesmo com a de fundo
        Cada anel tem 20 bolas"""
        self.goal = goal
        self.initial = initial
        esquerda, direita = initial
        self.topo=(3,17)
        self.fundo=(7,13)
        self.anel_index = {"esq": 0, "dir": 1}
        if len(esquerda)!=20:
            print('Anel esquerdo com número de bolas diferente de 20')
        elif len(direita)!=20:
            print('Anel direito com número de bolas diferente de 20')
        elif esquerda[self.topo[0]] != direita[self.topo[1]] or esquerda[self.fundo[0]] != direita[self.fundo[1]]:
             print("Há duas bolas que têm de ser partilhadas")
        else:
            contas=Counter("".join(esquerda)+"".join(direita))
            contas[direita[self.topo[1]]]-=1
            contas[direita[self.fundo[1]]]-=1
            valores=list(contas.values())
            valores.sort()
            if valores!=[9,9,10,10]:
                print("Temos de ter 4 cores com a lista de contas: [9,9,10,10]")
            else:
                self.cores={}
                for x in contas:
                    if contas[x] in self.cores:
                        self.cores[contas[x]]+=[x]
                    else:
                        self.cores[contas[x]] = [x]
        
    #def validate()
    
    def result(self, state, action):
        """The result"""
        (anel, sentido) = tuple(action.split())
        index_roda = self.anel_index[anel]
        index_outro = (index_roda + 1) % 2
        new_state = deepcopy(state)
        if sentido == "h":
            ult = new_state[index_roda].pop()
            new_state[index_roda].insert(0,ult)
        else:
            new_state[index_roda].append(new_state[index_roda][0])
            del new_state[index_roda][0]
        new_state[index_outro][self.topo[index_outro]]=new_state[index_roda][self.topo[index_roda]]
        new_state[index_outro][self.fundo[index_outro]]=new_state[index_roda][self.fundo[index_roda]]
        
        return new_state
    
    def actions(self,state):
        """The actions"""
        return ["esq h","esq ah", "dir h","dir ah"]
    
        
    
    def goal_ring(self,r):
        "The ring r satisfies the goal?"
        rr = r.copy()
        while rr[0]==rr[-1]:
            rr.insert(0,rr[-1])
            del rr[-1]
        ss = "".join(rr)
        #print("rodei-rodei:",ss)
        cor10_1,cor10_2 = self.cores[10]
        cor9_1, cor9_2 = self.cores[9]
        #print(ss.find(cor10_1*10))
        #print(ss.find(cor10_2*10))
        #print(ss.find(cor9_1*9))
        #print(ss.find(cor9_2*9))
        return (ss.find(cor10_1*10)!=-1 or ss.find(cor10_2*10)!=-1) and (ss.find(cor9_1*9)!=-1 or ss.find(cor9_2*9)!=-1)
    
    
    def goal_test(self,state):
        "If both the left and right rings satisfy the goal"
        return self.goal_ring(state[0]) and self.goal_ring(state[1])
    
    def display(self, state):
        """print the state"""
        header=""
        for i in range(len(state[0])):
            if i == self.topo[0]:
                header+="t"
            elif i == self.fundo[0]:
                header+='f'
            else:
                header+=" "
        print(header)
        print("".join(state[0]))
        header=""
        for i in range(len(state[1])):
            if i == self.topo[1]:
                header+="t"
            elif i == self.fundo[1]:
                header+='f'
            else:
                header+=" "
        print("".join(state[1]))
        print(header)
        print("")

def executa(p,estado,accoes):
    """ Executa uma sequênco greedyia de acções a partir do estado
        devolve um par (estado, custo)
    """
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    return (estado,custo)

## Teste da modelização em python

Vamos criar um problema válido com dois colares, com 'v' para verde, 'r' para roxo, 'e' para encarnado e 'a' para azul. E vamos mostrar usando o **display()**.

In [ ]:
anel_esq= list("vvvvvavvrreeeeeeeeev")
anel_dir= list("aaaaaaarrrrrrvvrrvva")
p = PuzzleColares(initial=Colaris((anel_esq,anel_dir)))
p.display(p.initial)

Verifiquemos se é estado final 

In [ ]:
p.goal_test(p.initial)

e quais as acções possíveis

In [ ]:
p.actions(p.initial)

Apliquemos uma acção: a rotação horária do segundo colar

In [ ]:
p.display(p.initial)
s1=p.result(p.initial,"dir h")
print("Depois de rodar o colar da direita no sentido horário")
p.display(s1)


Vamos confirmar que o estado original não mudou com a execução da acção

In [ ]:
p.display(p.initial)

Vamos aplicar a s1 a rotação anti-horária do mesmo colar e confirmemos que os estado inicial e o estado que resulta de uma rotação horária do colar da direita seguida de uma rotação anti-horária do mesmo colar são iguais, embora sejam objetos diferentes.

In [ ]:
s2=p.result(s1,('dir ah'))
s2 == p.initial

Criemos uma configuração que satisfaz o estado final

In [ ]:
anel_esq= list("vvvvvvvreeeeeeeeevvv")
anel_dir= list("aaaaaaarrrrrrrrrrvaa")
sf=Colaris((anel_esq,anel_dir))
p.display(sf)
print("Final?",p.goal_test(sf))

### Problemas Inválidos

#### Um dos anéis tem um número diferente de bolas

In [ ]:
anel_esq= list("vvvvavvrreeeeeeeeev")
anel_dir= list("aaaaaaarrrrrrvvrrvva")
erro = PuzzleColares(initial=(anel_esq,anel_dir))

#### Não partilham as bolas correctas

In [ ]:
anel_esq= list("vvvvvavvrreeeeeeeeev")
anel_dir= list("aaaaaaarrrrrrvvrvrva")
erro = PuzzleColares(initial=Colaris((anel_esq,anel_dir)))

#### As cores não são 4 ou não aparecem com o número de bolas correcto

In [ ]:
anel_esq= list("vvvvvvvreeeeeeeeevvv")
anel_dir= list("aaaaaaarrrrrrrrrvvaa")
erro = PuzzleColares(initial=Colaris((anel_esq,anel_dir)))

In [ ]:
anel_esq= list("vvvvvvvreeeeeeeeevvu")
anel_dir= list("aaaaaaarrrrrrrrrvvaa")
erro = PuzzleColares(initial=Colaris((anel_esq,anel_dir)))

### Teste de Procura de Solução
Vamos pegar no estado final de cima e aplicar-lhe uma sequência de algumas acções, não mais de 8 e a partir do estado resultante vamos criar um problema que será objeto do algoritmo de procura, sendo esse estado passado no construtor como estado inicial. Aplicaremos a procura em largura na sua versão em árvore. Se encontrar uma solução, a solução invertida e espelho da sequência usada, então tudo estará ok.

Por exemplo, se aplicarmos a sequência:
    
    'dir ah','dir ah','dir ah','esq h','esq h','esq h', 'dir ah' 
    
ao estado final, o problema a partir do estado resultante terá como solução: 

    'dir h', 'esq ah', 'esq ah', 'esq ah', 'dir h', 'dir h', 'dir h'

In [ ]:
print("Partindo do estado final anterior, vamos executar a seguinte sequência:")
sequencia=['dir ah','dir ah','dir ah','esq h','esq h','esq h', 'dir ah']
print(sequencia)
print()
ne=executa(p,sf,sequencia)
p.display(ne[0])


Vamos criar um novo problema que começa nessa configuração, a x passos do objectivo e vamos aplicar a procura em largura primeiro em árvore.

In [ ]:
p.initial=ne[0]
resultado=breadth_first_tree_search(p)
print("Solução Larg-prim (árvore) com custo", str(resultado.path_cost)+":")
print("Solução:",resultado.solution())
print("Profundidade:",resultado.depth)
nu=executa(p,ne[0],resultado.solution())
p.goal_test(nu[0])

## Comparando os vários algoritmos de largura

<img src="largura-primeiro-berkeley.PNG" alt="Drawing" style="width: 450px;"/>

Vamos comparar 3 algoritmos de procura em largura-primeiro, em termos de tempo e número de estados expandidos e dos custos de memória:
    
    Versão em árvore (teste do objectivo quando nó é seleccionado da fronteira e irá ser expandido)
    Versão em grafo (teste do objectivo quando nó é seleccionado da fronteira e irá ser expandido)
    Versão em grafo optimizada (teste do objetivo, quando se gera um novo nó e não quando vai ser expandido).

### Teste do objectivo quando se tenta expandir:
Sabemos que no pior caso a complexidade temporal em termos do número de estados expandidos é dependente do factor de ramificação b e da profundidade da solução, d. Com a versão tradicional em que o teste do objectivo é feito quando se tenta expandir o nó, usando a solução fica à profundidade m e na ponta mais à direita da árvore de procura, teremos

<img src="arvore.PNG" alt="Drawing" style="width: 450px;"/>

Notem que o estado solução nunca é considerado como expandido, daí a subtração por 1 na expressão.

No nosso caso, como temos um factor de ramificação de 4, porque temos sempre 4 acções possíveis, teremos um número de expandidos igual a 4<sup>0</sup>+4<sup>1</sup>+4<sup>2</sup>+...+4<sup>d</sup>-1. 

Notem que a complexidade espacial, dada pelo tamanho da fronteira quando é máxima é 4<sup>m+1</sup>-3. Isto porque quando queremos expandir o estado final, já todos os do seu nível (m) foram expandidos e todos os nós da camada m+1 foram gerados e estarão na fronteira juntamente com o objectivo e excepto os 4 sucessores do objectivo.

### Teste do objectivo nos sucessores
Há uma versão da largura que faz o teste de objectivo não ao nó que se expande mas aos seus sucessores, antecipando o processo de procura. Assim, no pior dos casos, em que a solução estivesse na ponta direita da árvore a uma profundidade m, haverá um nível inteiro de nós que não é necessário expandir, o nível maior, b<sup>m</sup>, dá-se uma enorme redução no número de nós a expandir: 4<sup>0</sup>+4<sup>1</sup>+4<sup>2</sup>+...+4<sup>d-1</sup>-1.

<img src="arvore2.PNG" alt="Drawing" style="width: 450px;"/>

Notem que a complexidade espacial, dada pelo tamanho da fronteira quando é máxima é 4<sup>m</sup>-3. A subtracção por 3 foi explicada em cima.

### Versão em grafo
Na versão em grafo, guardamos os estados gerados num conjunto e quando expandimos um nó, não geramos nós cujo estados tenham sido já gerados, i.e., pertençam a esse conjunto.

Nos problemas em que há muitos "caminhos que vão dar a Roma", isto é, em que há diferentes combinações de acções que levam ao mesmo estado e em que na árvore de procura os estados repetem-se com frequência, a versão em grafo é muito útil.
É precisamente o caso do puzze dos dois colares em que todas estas sequências de acções levam ao mesmo estado, neste caso ao próprio estado:
   
    dir h   dir ah
    dir ah  dir h
    dir ah  dir ah  dir h  dir h
    dir ah  dir h  dir ah  dir h
    ...
    
Vamos comparar a versão standard em árvore (teste de objectivo na expansão dos nós) com as 2 versões em grafo (teste de objectivo na expansão dos nós e teste de objectivo na geração dos nós)

Vamos ter as seguintes funções que executam a procura e recolhem o número de estados expandidos e do maior tamanho da fronteira ao longo do processo. Devolvem todos triplos em que o primeiro elemento é o nó da solução, o segundo é o número de expandidos e o terceiro é o maior tamanho da fronteira.

**breadth_first_tree_search_count()**: Versão em árvore (teste do objectivo quando nó é seleccionado da fronteira e irá ser expandido).

**breadth_first_graph_search_count()**: Versão em grafo (teste do objectivo quando nó é seleccionado da fronteira e irá ser expandido).

**breadth_first_graph_search_plus_count()**: Versão em grafo optimizada (teste do objetivo, quando se gera um novo nó e não quando vai ser expandido):

### O pior caso dos colares
No caso dos puzzle dos dois colares a ordem das acções é fixa e a última acção é rodar o colar direito no sentido anti-horário. Por isso, a sequência solução que seja o pior caso (a ponta direita da árvore de procura) a uma profundidade 4 será 'dir ah' 'dir ah' 'dir ah' 'dir ah'. Para criarmos o estado inicial que leva a essa solução pior teremos de aplicar ao estado final conhecido, a sequência inversa e espelho dessa: 'dir h' 'dir h' 'dir h' 'dir h'. E analogamente para o caso de N acções. 
No entanto, para mais do que 10 movimentos esta forma de gerar os piores casos já não funciona porque 12 movimentos anti-horários podem ser traduzidos por 8 horários.  

A função seguinte, gera o estado inicial que devolve a solução correspondente ao pior dos casos a uma profundidade N e aplica o algoritmo alg para procurar a solução e contar.
Conta também o tempo e mostra todos os resultados, confirmando que o estado final que encontra é o mesmo que usámos para gerar o estado inicial do problema.

In [ ]:
def testa_pior(p,sf,N,alg,funcao_h=None):
    """Nesta função vamos gerar a solução mais à direita com profundidade N. 
    A acção 'dir ah' é a última da lista, logo a solução mais à direita com profundidade N é N vezes 'dir ah'.
    Pegamos num estado final e aplicamos a sequência de acções espelho dessa: N vezes 'dir h'.
    Construímos um problema a partir do estado resultante e aplicamos a procura em largura primeiro em árvore, contando o
    tempo que demora e o número de estados expandidos. Como a solução é a mais à direita, a árvore completa é gerada e todos os 
    estados até ao nível N-1 são expandidos. Para o caso da solução à profundidade 5 expandimos 4^0+4^1+4^2+4^3+4^4"""
    p.initial=executa(p,sf,['dir h']*N)[0]
    start = timeit.default_timer()
    if funcao_h:
        resultado,expanded,max_space=alg(p,funcao_h)
    else:
        resultado,expanded,max_space=alg(p)
    stop = timeit.default_timer()
    timeTree = stop-start
    print("Solução com custo", str(resultado.path_cost)+":")
    print("Solução:",resultado.solution())
    print("Profundidade:",resultado.depth)
    nu=executa(p,p.initial,resultado.solution())
    print('TEMPO GASTO:',timeTree)
    print('Expandidos:',expanded)
    print('Tamanho Máximo da fronteira:',max_space)
    print('The same?',sf==nu[0])

Vamos então aplicar o **testa_pior()** para a largura-primeiro standard em árvore para o caso da pior solução a uma profundidade 4. Notem que já tínhamos guardado o estado final da figura inicial na variável sf.

In [ ]:
testa_pior(p,sf,4,breadth_first_tree_search_count)

Podemos confirmar que a soma de potências dá o mesmo número, fazendo uma função que recebe a ramificação e a profundidade e devolve o resultado do número de estados expandidos.

In [ ]:
def somaPotencias(ram,prof):
    soma=0
    out=str(ram)+"^0"
    for i in range(1,1+prof):
        out+="+"+str(ram)+"^"+str(i)
        soma+=pow(ram,i)
    return out+'-1='+str(soma)

print('Os expandidos terão de ser',somaPotencias(4,4))
print('Os gerados serão 4^5-3=',4**5-3)

Para verem como a complexidade temporal e a espacial crescem podemos ver que para uma profundidade de 10 teremos perto de 1 milhão e meio de expandidos...

In [ ]:
print(somaPotencias(4,10))

e teremos uma fronteira máxima com um pouco mais de 4 milhões de nós dada por 4<sup>11</sup>-3

In [ ]:
print('Os gerados serão 4^11-3=',4**11-3)

### Comparação dos algoritmos de Largura-primeiro para soluções de 7 e 8 acções
Comparem os três algoritmos para o pior caso para as sequências de 7 e 8 acções. 

In [ ]:
# solução
print('\n Pior à profundidade 7 - largura em árvore')
testa_pior(p,sf,7,breadth_first_tree_search_count)
print('\n Pior à profundidade 7 - largura em grafo')
testa_pior(p,sf,7,breadth_first_graph_search_count)
print('\n Pior à profundidade 7 - largura em grafo look ahead')
testa_pior(p,sf,7,breadth_first_graph_search_plus_count)
print('\n Pior à profundidade 8 - largura em árvore')
testa_pior(p,sf,8,breadth_first_tree_search_count)
print('\n Pior à profundidade 8 - largura em grafo')
testa_pior(p,sf,8,breadth_first_graph_search_count)
print('\n Pior à profundidade 8 - largura em grafo look ahead')
testa_pior(p,sf,8,breadth_first_graph_search_plus_count)

### Comparação dos algoritmos para soluções de 9 e 10 acções
Comparem os dois algoritmos de largura-primeiro em grafo para o pior caso para a sequência de 9 e de 10 acções. Não vamos utilizar a versão em árvore porque "it is too much"!

In [ ]:
print('\n Pior à profundidade 9 - largura em grafo')
testa_pior(p,sf,9,breadth_first_graph_search_count)
print('\n Pior à profundidade 9 - largura em grafo look ahead')
testa_pior(p,sf,9,breadth_first_graph_search_plus_count)
print('\n Pior à profundidade 10 - largura em grafo')
testa_pior(p,sf,10,breadth_first_graph_search_count)
print('\n Pior à profundidade 10 - largura em grafo look ahead')
testa_pior(p,sf,10,breadth_first_graph_search_plus_count)

## Greedy, A* e heurísticas

Vamos usar como heurística uma função que conta o número de sequências ininterruptas em ambos os colares e que subtrai 6. Notem que a solução tem 3 sequências em cada colar.

### Função heurística h1()
Implementem a função heurística a que podem chamar de **h1()**. Tradicionalmente convém colocar a função heurística dentro da classe, como um dos seus métodos. Por uma questão de não mexer na classe e para uma avaliação automática deste desafio não o vamos fazer.

In [ ]:
# solução

def h1(no):
    def num_seq(lista):
        out = 1
        pre = lista[0]
        for x in lista[1:]:
            if x != pre:
                out+=1
            pre=x
        if lista[0]==lista[-1]:
            out-=1
        return out

    esquerdo,direito=no.state
    return num_seq(esquerdo)+num_seq(direito)-6



### Teste de h1()
Testem esta função heurística no puzzle da imagem a seguir:

<img src="inicialDoisColares.PNG" alt="Drawing" style="width: 250px;"/>

In [ ]:
anel_esq= list("vvvvvavvrreeeeeeeeev")
anel_dir= list("aaaaaaarrrrrrvvrrvva")
estado=Colaris((anel_esq,anel_dir))
p=PuzzleColares(estado)
print('Heurística do estado:',h1(Node(estado)))

A função em baixo aplica um algoritmo heurístico (alg) e a sua função heurística (funcao_h) a uma situação do tipo pior caso, gerada a partir de N rotações horárias do colar da direita. Devolve a solução, o tempo da procura e o número de expandidos. 

### Comparando o Greedy e o A*, usando h1(), para o pior caso a 10 de distância
Usem a função **testa_pior()** para o Greedy-melhor-primeiro e o A* nas suas versões em grafo e de recolha de dados da procura, respectivamente **astar_search_count()** e **greedy_best_first_graph_search_count()**, usando a função heurística **h1()** para o worst case de 10 acções.

In [ ]:
print('\n Pior à profundidade 10 - A* em grafo (h1)')
testa_pior(p,sf,10,astar_search_count,h1)
print('\n Pior à profundidade 10 - Greedy em grafo (h1)')
testa_pior(p,sf,10,greedy_best_first_graph_search_count,h1)

Notem que o Greedy esteve fantástico com esta heurística para os estados iniciais a 9 e 10 aplicando sempre rotações horárias a partir do estado final. O A* também esteve muito bem comparado com as Larguras.

### Gerando problemas através da aplicação de sequências de acções a partir do estado final

A função seguinte aplica uma sequência de acções ao estado (sf) e depois constrói um problema com o estado inicial resultante, aplicando o algoritmo de procura alg.

In [ ]:
def testa_seq(p,sf,accoes,alg,funcao_h=None):
    """"""
    p.initial=executa(p,sf,accoes)[0]
    start = timeit.default_timer()
    if funcao_h:
        resultado,expanded,max_space=alg(p,funcao_h)
    else:
        resultado,expanded,max_space=alg(p)
    stop = timeit.default_timer()
    timeTree = stop-start
    print("Solução com custo", str(resultado.path_cost)+":")
    print("Solução:",resultado.solution())
    print("Profundidade:",resultado.depth)
    nu=executa(p,p.initial,resultado.solution())
    print('TEMPO GASTO:',timeTree)
    print('Expandidos:',expanded)
    print('Tamanho Máximo da fronteira:',max_space)
    print('Atingimos o mesmo final?',sf==nu[0])

###### Vamos comparar os algoritmos de procura em largura em grafo com o A* e Greedy para soluções de tamanho 12 geradas a partir do estado final (que temos usado) após aplicação das seguintes acções:

```python
['esq ah', 'esq ah', 'dir ah', 'esq h', 'dir ah', 'dir ah', 'dir ah', 'dir ah', 'dir ah', 'esq h', 'dir h', 'esq ah']
```
Não vale a pena fazer a procura em largura na sua variante árvore e teste de objectivo na expansão porque o número de estados expandidos seria muito elevada. Notem que como a solução está a uma profundidade de 12, isso quer dizer que pelo menos todo o nível de 11 tem de ser expandido (5,592404 milhões de nós) e no pior caso termos de expandir todos menos 1 do nível 12 (22,369620 milhões de nós). É claro que a sequência de acções não é a pior possível, e sendo assim, o número de expandidos ficará entre esses dois valores. Se fosse a pior possível o número de nós gerados seria de 67,108863 milhões, mas não é o caso, ficando por um valor entre 22,4 e 67,1 milhões.

In [ ]:
print('Os expandidos terão de ser mais do que ',somaPotencias(4,11),'e menos do que ',somaPotencias(4,12))

print('Os gerados terão de ser mais do que  4^12-3=',4**12-3,'e menos do que  4^13-3=',4**13-3)

### Comparação de problema a 12 passos da solução

Comecemos por comparar as larguras em grafo para duas sequências à distância de 12 da solução:

In [ ]:
accoes12 = ['esq ah', 'esq ah', 'dir ah', 'esq h', 'dir ah', 'dir ah', 'dir ah', 'dir ah', 'dir ah', 'esq h', 'dir h', 'esq ah']
print('\n Pior à profundidade 12 - largura em grafo')
testa_seq(p,sf,accoes12,breadth_first_graph_search_count)
print('\n Pior à profundidade 12 - largura em grafo look ahead')
testa_seq(p,sf,accoes12,breadth_first_graph_search_plus_count)

Comparemos com o A* com h1() para a mesma configuração a 12 passos:

In [ ]:
print('\n Solução à profundidade 12 - A* em grafo (h1)')
testa_seq(p,sf,accoes12,astar_search_count,h1)

E o Greedy? Vamos tentar, mas um pouco desconfiados vamos dar um limite de tempo para o fazer: 300s, ou seja, 5m. Para isso vamos usar funções do módulo *func_timeout*. Atenção que talvez tenham que instalar o módulo *func_timeout*, escrevendo *pip func_timeout* no *anaconda prompt*, que podem encontrar na lupa do windows.

In [ ]:
print('\n Solução à profundidade 12 - Greedy em grafo (h1)')
from func_timeout import func_timeout, FunctionTimedOut

try:
    ReturnedValue = func_timeout(300, testa_seq, args=(p,sf,accoes12,greedy_best_first_graph_search_count,h1))
except FunctionTimedOut:
    ReturnedValue='Ui UI Não tenho mais paciência!'
    print('Não encontrámos a solução em 300s')
ReturnedValue 

### Soluções a 13 passos

Vamos agora comparar as duas variantes da procura em largura-primeiro em grafo com o A*  para soluções de tamanho 13 geradas a partir do estado final após aplicação das seguintes acções:

```python
['esq h', 'dir ah', 'dir ah', 'esq h', 'esq h', 'esq h', 'esq h', 'dir ah', 'dir ah', 'esq h', 'dir h', 'esq ah', 'esq ah']
```

In [ ]:
accoes13 =['esq h', 'dir ah', 'dir ah', 'esq h', 'esq h', 'esq h', 'esq h', 'dir ah', 'dir ah', 'esq h', 'dir h', 'esq ah', 'esq ah']
print('\n Pior à profundidade 13 - largura em grafo')
testa_seq(p,sf,accoes13,breadth_first_graph_search_count)
print('\n Pior à profundidade 13 - largura em grafo look ahead')
testa_seq(p,sf,accoes13,breadth_first_graph_search_plus_count)

Já não arriscamos usar o Greedy devido ao falhanço com soluções de 12 acções em menos de 5m. Apliquemos apenas o A*.

In [ ]:
print('\n Solução à profundidade 13 - A* em grafo (h1)')
testa_seq(p,sf,accoes13,astar_search_count,h1)

O A* não deu a solução óptima.  A solução devolvida é de tamanho 15 e não 13 que foi a que resultou da procura em largura que é optimal neste problema de custos homogéneos. O A* demorou mais tempo também, embora tenha expandido muito menos nós. Surpreendidos?

### h1() é consistente?
A heurística referida não pode ser consistente pois essa propriedade era garantia da optimalidade! Porque é que não é consistente?

Sabemos que se não é admissível também não é consistente. Provemos então que não é admissível e que por derivação não é também consistente.

Não é admissível porque podemos num só movimento atingir o objectivo numa situação em que o valor heurístico é de 2.
Eis o exemplo do estado final que tem 0 de valor heuristico:

Confirmemos a sua não admissibilidade e também a não consistência.

In [ ]:
# Confirmação da não admissibilidade e da não consistência

p.display(sf)
print('Heurística?',h1(Node(sf)))

Vamos aplicar-lhe uma acção: 'dir h'

Obtemos um estado que está à distância de 1 acção do objectivo: a rotação espelho de 'dir h', i.e., 'dir ah', mas o seu valor heurístico é de 2, violando a admissibilidade.



In [ ]:
print('Depois de aplicarmos a rotação horária do colar direito')
x=p.result(sf,'dir h')
p.display(x)
print('Heurística?',h1(Node(x)))

A heurística do estado x é 2, sendo na realidade a menor distância apenas de uma acção (sobrestimando o menor custo ao objectivo), por isso não admissível; 

Poderíamos atacar logo a falha na consistência.

Para o mesmo estado sf e x, h1(x)-h1(sf)=2 e o custo entre x e sf é apenas de uma acção, violando a consistência.

### Segunda função heurística h2(x)=h1(x)/2
Vamos dividir a função h1(x) por 2. Esta é admissível e consistente. Na verdade, quando rodamos um colar, esse colar não muda em termos do número de sequências ininterruptas. O outro colar é que pode mudar e na melhor das hipóteses, 3 sequências interrompidas nas bolas partilhadas podem transformar-se numa só sequência, no melhor dos casos, podendo haver no máximo uma diminuição de 2 sequências em ambos os colares (um não muda), o que quer dizer que devemos dividir h1(x) por 2 (3-1).

In [ ]:
def h2(no):
    return h1(no)/2
    
anel_esq= list("vvvvvavvrreeeeeeeeev")
anel_dir= list("aaaaaaarrrrrrvvrrvva")
p = PuzzleColares(initial=Colaris((anel_esq,anel_dir)))

Vamos então verificar se encontra a solução óptima para a solução das 13 acções usada em cima. 

In [ ]:
print('\n Solução à profundidade 13 - A* em grafo (h2)')
testa_seq(p,sf,accoes13,astar_search_count,h2)

Na verdade, encontrámos a solução óptima mas o A* continua a demorar muito mais tempo a encontrá-la do que as duas versões da largura primeiro, e expandindo muito menos nós. Porque será?
Achamos que deve ser porque o teste de estados que se encontram na fronteira é caro computacionalmente, por ser feito numa lista, ao contrário do teste de expandidos, e a fronteira pode ser enorme. Soma-se a isso o custo de calcular o valor heurístico e a inserção dos nós na lista de prioridades que é a fronteira.

### Soluções a 14 passos

Avancemos para a comparação de soluções a 14 passos... mas apenas com a largura porque o A* mostrou ser pouco eficiente.

In [ ]:
accoes14 =['esq h', 'dir ah', 'dir ah', 'esq h', 'esq h', 'esq h', 'esq h', 'dir ah', 'dir ah', 'esq h', 'dir h', 'esq ah', 'esq ah','esq ah']
print('\n Pior à profundidade 14 - largura em grafo')
testa_seq(p,sf,accoes14,breadth_first_graph_search_count)
print('\n Pior à profundidade 14 - largura em grafo look ahead')
testa_seq(p,sf,accoes14,breadth_first_graph_search_plus_count)
# testa_seq(p,sf,accoes14,astar_search_count,h2)

## Exercício
Implemente a seguinte heurística: devolve a soma para cada colar da seguinte função: 10-tamanho-maior + 9 - tamanho-segunda-maior.
tamanho_maior e tamanho-segunda-maior são respetivamente o tamanho da maior e da segunda maior sequências ininterruptas de cores no colar.
No caso da solução final teremos em cada colar 10-10+9-9=0 e ambos colares 0+0=0.
Testa-a para os estados que resultam de aplicar as 14 e as 16 acções seguintes ao estado final sf:

```python

accoes14=['esq h', 'dir ah', 'dir ah', 'esq h', 'esq h', 'esq h', 'esq h', 'dir ah', 'dir ah', 'esq h', 'dir h', 'esq ah', 'esq ah','esq ah']
accoes16=['dir h', 'dir h',  'dir h', 'esq h', 'dir ah', 'esq ah', 'dir ah', 'esq ah', 'dir ah', 'esq h', 'esq h', 'dir ah', 'esq h', 'dir h', 'esq h', 'dir h']
```

e depois aplica os algoritmos do A* com a contagem do tempo, dos expandidos e do maior tamanho da fronteira.

In [ ]:
# h3




# Prazo de Entrega

Final do semestre, último dia de aulas, 18 de Dezembro à meia noite menos 1 m.

# Submissão
A informação sobre o processo de submissão sairá muito em breve.